$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [50]:
import pandas as pd

def analyze(dataframe: pd.DataFrame, lcol: int = 0, rcol: int = 0, is_general: bool = False):
    """
    Get averages and #bests of `dataframe` of results from betas experiment.

    `lcol`: left column index of differences, beta = 0

    `rcol`: right column index of differences, beta = -1

    `is_general`: `True` if `dataframe` is a general table of all instances,
    #bests are counts of minimum averages;
    otherwise #bests are counts of 0.
    """
    # get columns of differences
    diffs = dataframe if is_general else dataframe.iloc[:, lcol:rcol]
    avgs = diffs.mean()

    other = diffs.min(axis=1) if is_general else 0
    bests = diffs.eq(other, axis=0).sum()

    data = {
        "average": avgs,
        "#bests": bests,
    }

    return pd.DataFrame(data.values(), index=data.keys())

In [51]:
do_save = False
do_display_each = True

In [55]:
from IPython.display import display
from grasp_betas import read_results

BOLD_PROP = "font-weight:bold;"

datalist = []

names = [
    # "rd400_267_133",
    # "rat783_522_261", # 1
    # "nrw1379_920_459",
    "anpcp_882_441_t5.0",
]
for name in names:
    betas = read_results(name)
    
    if do_display_each:
        print(name)

    for p in betas["p"].unique():
        for alpha in betas["alpha"].unique():
            # get dataframe with p and alpha fixed
            fixed = betas[(betas["p"] == p) & (betas["alpha"] == alpha)]
            # add multiindex
            concated = pd.concat(
                {
                    "params": fixed.iloc[:, 0:3],
                    "OF": fixed.iloc[:, 3:10],
                    "diffs": fixed.iloc[:, 10:17],
                    "time": fixed.iloc[:, 17:24],
                    "last_imp": fixed.iloc[:, 24:31],
                },
                axis=1,
            )
            
            analysis = analyze(fixed, 10, 17)
            styled_analysis = (
                analysis.style
                    .highlight_min(("average", analysis.columns), axis=1, props=BOLD_PROP)
                    .highlight_max(("#bests", analysis.columns), axis=1, props=BOLD_PROP)
            )
            
            if do_display_each:
                display(concated.style.highlight_min(concated.columns[3:10], axis=1, props=BOLD_PROP))
                display(styled_analysis)

            row = [name, p, alpha]
            # get all averages of this combination
            row.extend(analysis.iloc[0, :])
            datalist.append(row)

params = "instance p alpha".split()
headers = params + analysis.columns.tolist()
# dataframe of all tests
table = pd.DataFrame(datalist, columns=headers)

# group table so params become indexes of dataframe, instead of columns
gpdtable = table.groupby(params, sort=False).mean()
styled_gpdtable = gpdtable.style.highlight_min(axis=1, props=BOLD_PROP)
display(styled_gpdtable)

analysis = analyze(gpdtable, is_general=True)
display(
    analysis.style
        .highlight_min(("average", analysis.columns), axis=1, props=BOLD_PROP)
        .highlight_max(("#bests", analysis.columns), axis=1, props=BOLD_PROP)
)

anpcp_882_441_t5.0


,0.0,0.2,0.4,0.6,0.8,1.0,-1
average,4.551661,1.684115,1.411015,0.000000,1.365498,5.871643,1.274465
#bests,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000


,,,0.0,0.2,0.4,0.6,0.8,1.0,-1
instance,p,alpha,,,,,,,
anpcp_882_441_t5.0,44,2,4.551661,1.684115,1.411015,0.000000,1.365498,5.871643,1.274465


,0.0,0.2,0.4,0.6,0.8,1.0,-1
average,4.551661,1.684115,1.411015,0.000000,1.365498,5.871643,1.274465
#bests,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000


In [53]:
# styled_gpdtable.format(precision=2).to_latex("styled_gpdt.tex", convert_css=True)

In [54]:
# gpdtable.to_latex("gpdt.tex", float_format="%.2f", multirow=True)